<a href="https://colab.research.google.com/github/kangwonlee/eng-math-2/blob/main/Ch12_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym



### Example 12.3.1



In [ ]:
mx = 5

fig = plt.figure(figsize=(6, mx*2))

for n in range(1, mx):
    plt.subplot(mx, 1, n)

    x = np.linspace(0, 2+1e-7, 361)
    y = np.cos((n * np.pi * 0.5) * x)
    plt.fill_between(x, y, label=fr'$\cos \frac{{{n}}}{{2}}\pi x$')

    plt.xlabel('$x$')
    plt.legend(loc=0)
    plt.xlim(0, 2)
    plt.grid(True)

plt.tight_layout()



In [ ]:
def calc_f(x_rad):
    result = np.zeros_like(x_rad)
    x_phase = x_rad % (4)
    result[x_phase<2] = x_phase[x_phase<2]
    result[x_phase>=2] = x_phase[x_phase>=2] - 4
    return result


def calc_an(n):
    return 0.0


def calc_bn(n):
    return 4.0 * (-1)**(n+1) /(n * np.pi)


def calc_mode_n(x, p_inv, n):
    bn = calc_bn(n)
    n_pi_pinv = n*np.pi* p_inv
    x_inside = n_pi_pinv * x 
    mode_n = bn * np.sin(x_inside)
    return mode_n


def calc_Sn(n_sigma, x, p=np.pi):
    p_inv = 1.0 / p
    result = np.zeros_like(x)
    for n in range(1, n_sigma+1):
        mode_n = calc_mode_n(x, p_inv, n)
        result += mode_n
    return result



In [ ]:
N = 10
p = 2.0
pinv = 1.0 / p

x = np.arange(-10, 10+1e-7,0.01)
x_vlines = np.arange(x.min(), x.max()+1e-7, 2.0)

fig, axs = plt.subplots(N-1, 2, figsize=(12, 1.5*(N-1)))

axs_mode = axs[:,0]
axs_fourier = axs[:,1]

for n_sigma, ax_mode, ax_fourier in zip(range(1, N), axs_mode, axs_fourier):
    ax_mode.plot(x, calc_mode_n(x, pinv, n_sigma))

    ax_fourier.plot(x, calc_f(x))
    ax_fourier.vlines(x_vlines, -2, 2, linestyles='dotted')
    y_fourier = calc_Sn(n_sigma, x, p)
    ax_fourier.plot(x, y_fourier)

    ax_mode.set_xlim(x.min(), x.max())
    ax_mode.set_ylim(-2, 2)
    ax_fourier.set_xlim(x.min(), x.max())
    
    ax_mode.grid(True)
    ax_fourier.grid(True)

plt.tight_layout()



### Spectrum



In [ ]:
freq_rad_list = [0]

a_list = [calc_an(0)]
b_list = [0]

mag_list = [abs(calc_an(0))]
arg_rad_list = [0.0]

for n in range(1, N):
    an = calc_an(n)
    bn = calc_bn(n)

    freq_rad_list.append(n*np.pi/p)
    a_list.append(an)
    b_list.append(bn)

    mag_list.append((an**2 + bn**2)**0.5)
    arg_rad_list.append(np.arctan2(-bn, an))

# wrap phase to avoid excessive discontinuity
arg_rad_array = np.array(arg_rad_list)
arg_deg_array = np.rad2deg(arg_rad_list)

freq_deg_array = np.array(freq_rad_list) * (0.5 / np.pi)

plt.close()
plt.subplot(2, 1, 1)
plt.plot(freq_deg_array, mag_list, '.-')
plt.ylabel('magnitude')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(freq_deg_array, arg_deg_array, '.-')
plt.xlabel('freq(Hz)')
plt.ylabel('phase(deg)')
plt.grid(True)
plt.tight_layout()



### Example 12.3.2



In [ ]:
def calc_f(x_rad):
    result = np.ones_like(x_rad)
    x_phase = x_rad % (2 * np.pi)
    result[x_phase>=np.pi] = -1.0
    return result


def calc_an(n):
    return 0.0


def calc_bn(n):
    return 2.0 * (1-(-1)**(n)) /(n * np.pi)


def calc_mode_n(x, p_inv, n):
    bn = calc_bn(n)
    n_pi_pinv = n*np.pi* p_inv
    x_inside = n_pi_pinv * x
    mode_n = bn * np.sin(x_inside)
    return mode_n


def calc_Sn(n_sigma, x, p=np.pi):
    p_inv = 1.0 / p
    result = np.zeros_like(x)
    for n in range(1, n_sigma+1):
        mode_n = calc_mode_n(x, p_inv, n)
        result += mode_n
    return result



In [ ]:
N = 10

x = np.arange(-10, 10+1e-7,0.01)
x_vlines = np.arange(-3, 3+1e-7, 1.0) * np.pi

fig, axs = plt.subplots(N-1, 2, figsize=(12, 1.5*(N-1)))

axs_mode = axs[:,0]
axs_fourier = axs[:,1]

for n_sigma, ax_mode, ax_fourier in zip(range(1, N), axs_mode, axs_fourier):
    ax_mode.plot(x, calc_mode_n(x, 1.0/np.pi, n_sigma))

    ax_fourier.plot(x, calc_f(x))
    ax_fourier.vlines(x_vlines, -2, 2, linestyles='dotted')
    y_fourier = calc_Sn(n_sigma, x, np.pi)
    ax_fourier.plot(x, y_fourier)

    ax_mode.set_xlim(x.min(), x.max())
    ax_mode.set_ylim(-2, 2)
    ax_fourier.set_xlim(x.min(), x.max())
    
    ax_mode.grid(True)
    ax_fourier.grid(True)

plt.tight_layout()



### Spectrum



In [ ]:
freq_rad_list = [0]

a_list = [calc_an(0)]
b_list = [0]

mag_list = [abs(calc_an(0))]
arg_rad_list = [0.0]

for n in range(1, N):
    an = calc_an(n)
    bn = calc_bn(n)

    freq_rad_list.append(n*np.pi/p)
    a_list.append(an)
    b_list.append(bn)

    mag_list.append((an**2 + bn**2)**0.5)
    arg_rad_list.append(np.arctan2(-bn, an))

# wrap phase to avoid excessive discontinuity
arg_rad_array = np.array(arg_rad_list)
arg_deg_array = np.rad2deg(arg_rad_array)
arg_diff_deg_array = np.hstack([[0], np.diff(arg_deg_array)])
arg_deg_array[arg_diff_deg_array>175] += (-180)

freq_deg_array = np.array(freq_rad_list) * (0.5 / np.pi)

plt.close()
plt.subplot(2, 1, 1)
plt.plot(freq_deg_array, mag_list, '.-')
plt.ylabel('magnitude')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(freq_deg_array, arg_deg_array, '.-')
plt.xlabel('freq(Hz)')
plt.ylabel('phase(deg)')
plt.grid(True)
plt.tight_layout()



## Example 12.3.4



$$
b_n
=
  2\int_{0}^{1}
    \pi t \sin n \pi t dt
$$



$$
x_p(t)
=
  \sum_{n=1}^{\infty}
    \frac{2(-1)^{n+1}}
      {n\left[k-mn^2\pi^2\right]}
    \sin n\pi t
$$



In [ ]:
n = sym.symbols('n', positive=True, integer=True)
t = sym.symbols('t')

bn = sym.simplify(
    2 * sym.integrate(
        sym.pi * t * sym.sin(n * sym.pi * t),
        (t, 0, 1)
    )
)
bn



In [ ]:
m = 1 / 16.0
k = 4.0

def calc_bn(n, m=(1/16), k=4):
    return (
        (2 * (-1)**(n+1))
        /(n * (k - m * n * n * np.pi * np.pi))
    )


def calc_mode(n, x, m=(1/16), k=4):
    return calc_bn(n, m, k) * np.sin(n * np.pi * x)



In [ ]:
N = 10

x = np.arange(0, 5+1e-7, 0.01)
y = np.zeros_like(x)

for n in range(1, N+1):
    mode = calc_mode(n, x, m, k)
    y += mode

plt.plot(x, y)
plt.xlabel('t')
plt.ylabel('$x_p(t)$')
plt.xlim((x.min(), x.max()))
plt.grid(True)

